## 1. 필요한 라이브러리 불러오기

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn.init as init
import time

## 2. 하이퍼 파라미터 및 gpu 사용

In [2]:
# Hyper Parameters
IMAGE_SIZE = 224
LEARNING_RATE = 0.01
epochs = 20
batch_size = 128

# GPU
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(device)

2.9.0+cu126
cuda


## 3. 데이터 준비

In [3]:
# 데이터 준비
cifar_mean = [0.4914, 0.4822, 0.4465]
cifar_std = [0.2023, 0.1994, 0.2010]

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean, cifar_std)
])

test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean, cifar_std)
])

train_data = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_data = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=test_transform
)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:03<00:00, 47.3MB/s]


In [4]:
# 데이터 형상 확인
images, labels = next(iter(train_dataloader))
print(images[0].shape)

torch.Size([3, 224, 224])


## 4. Block 단위 클래스

In [5]:
# block 단위 클래스 정의
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # Conv1 => 다운샘플링 역할을 맡을 수 있음(stride=2 or 1)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        # Conv2 => 특징 추출(항상 stride=1)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        # Conv 이후 BN을 적용하면 bias는 상쇄되므로 bias=False로 지정

        # Projection Shortcut or Identity mapping
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        # F(x) => Conv1-BN-ReLU-Conv2-BN
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        # F(x) + x
        out += self.shortcut(x)

        # ReLU(F(x) + x)
        out = F.relu(out)
        return out

## 5. ResNet 클래스

In [ ]:
# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        # 채널 수
        self.in_planes = 64

        # Stem (입력부)
        # 이미지의 크기를 줄이는 과정을 진행
        # 224x224 -> 7x7conv1(112x112) -> 3x3MaxPool(56x56)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Layers (층)
        # 각 Layer에서 사용할 conv의 개수 정의
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) # Stem 부분에서 충분히 다운샘플링 진행했기때문에 stride=1로 지정
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Head (출력부)
        # AdaptiveAvgPooling기  법을 통해 1x1크기로 변환후 펼쳐서 완전연결(fc)해준다.
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        # 첫 번쨰 stride=2로 하여 크기를 줄이고
        # 두 번쨰 stride=1로 하여 형상을 유지
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            # block을 생성하여 layer에 담음
            layers.append(block(self.in_planes, planes, stride))
            # 채널 수 변경
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out) # 224->56

        out = self.layer1(out) # 56->56
        out = self.layer2(out) # 56->28
        out = self.layer3(out) # 28->14
        out = self.layer4(out) # 14->7

        out = self.avgpool(out) # 7x7->1x1
        out = torch.flatten(out, 1) # 1차원으로 flatten
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2]) # 1 + 2*2*4 + 1 = 18 layers

## 6. 훈련 준비 및 optimizer 선언

In [7]:
# 훈련 준비
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
# 최적화 => SGD + Momentum
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)

## 7. 훈련

In [ ]:
from google.colab import drive

# 구글 마운트 및 경로 설정
drive.mount('/content/drive')
save_dir = '/content/drive/{원하는 경로}'
os.makedirs(save_dir, exist_ok=True)

# ResNet 학습
print(f"학습 시작")
best_acc = 0
for epoch in range(epochs):
    start_time = time.time()

    # Training
    model.train()
    running_loss = 0.0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_acc = 100 * correct / total
    epoch_time = time.time() - start_time

    # Best Model 저장 로직
    if epoch_acc > best_acc:
        best_acc = epoch_acc

        # 구글 드라이브 경로에 저장
        save_path = os.path.join(save_dir, f"resnet18_best_{epoch_acc:.0f}.pth")
        torch.save(model.state_dict(), save_path)

    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"Loss: {running_loss/len(train_dataloader):.3f} | "
          f"Test Acc: {epoch_acc:.2f}% | "
          f"Time: {epoch_time:.1f}s")

print("학습 종료")